In [ ]:
import numpy as np
from keras.models import load_model
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')

import os
import tensorflow as tf
import keras
import cv2
from keras import regularizers
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers , models, optimizers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50V2
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPool2D, Bidirectional,AveragePooling2D, LSTM, Reshape ,Input, BatchNormalization, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing import image
import scipy
import os
import cv2
train_dir = '../input/fer2013/train/'
test_dir = '../input/fer2013/test/'



In [ ]:
train_datagen =  ImageDataGenerator(
        rescale = 1 / 255.,
        rotation_range=10,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,                                        
        fill_mode='nearest',
    )
train_set = train_datagen.flow_from_directory(train_dir,target_size = (48,48),
                                                color_mode = 'grayscale',
                                             batch_size = 64,
                                             class_mode = 'categorical',
                                            
                                             shuffle = False)

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,target_size = (48,48),
                                                color_mode = 'grayscale',
                                             batch_size = 64,
                                             class_mode = 'categorical',
                                            
                                             shuffle = False)

In [ ]:
x_train, y_train = next(train_set)
x_val, y_val = next(test_set)

y_train_sparse = np.argmax(y_train, axis = 1)
y_val_sparse = np.argmax(y_val, axis = 1)



In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes, normalize=False, cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):

            color = 'white' if cm[i, j] > thresh else 'black'

            plt.text(j, i, cm[i, j], ha='center', va='center', color=color)


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
def generateModel(activation):
    model = Sequential([
        Input((48, 48, 1)),
        Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid'),
        BatchNormalization(axis=3),
        Activation(activation),
        MaxPooling2D((2,2)),
        Flatten(),
        Dropout(0.5),
        Flatten(),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    model.summary()
    adam = keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam

param_grid = {
    'activation': ['tanh', 'relu'],
}

grid_search = GridSearchCV(estimator=KerasClassifier(generateModel), param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)
import pandas as pd
print(pd.DataFrame(grid_search.cv_results_))

In [ ]:
model = generateModel('relu')
steps_per_epoch = train_set.n // train_set.batch_size
validation_steps = test_set.n // test_set.batch_size

In [ ]:
history = model.fit(train_set ,validation_data = test_set , epochs=50, batch_size=64
                                      )


In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy plot")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("emotion_model_accuracy.png")

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Accuracy loss")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig("emotion_model_loss.png")

In [ ]:
predictions = model.predict(test_set)
pred = np.argmax(predictions, axis = 1)

print(accuracy_score(test_set.classes,pred))

cm = confusion_matrix(test_set.classes, pred)
class_names = test_set.class_indices.keys()
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cm, classes=class_names)
plt.show()

In [ ]:
model = Sequential([
    Input((48, 48, 1)),
    Conv2D(16, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    Conv2D(32, (3,3), strides=(1,1), padding = 'same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    Conv2D(32, (3,3), strides=(1,1), padding = 'valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation = 'softmax')
])

model.summary()

In [ ]:
adam = keras.optimizers.Adam(learning_rate=0.0002)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
steps_per_epoch = train_set.n // train_set.batch_size
validation_steps = test_set.n // test_set.batch_size

In [ ]:
history = model.fit(train_set ,validation_data = test_set , epochs=50, batch_size=64,
                                      steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy plot")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("emotion_model_accuracy.png")

The accuracy was increasing in a perfect manner, and there is no overfitting. We will run this model for 100 epochs the next time, and see how things look.

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Accuracy loss")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig("emotion_model_loss.png")

In [ ]:
predictions = model.predict(test_set)
pred = np.argmax(predictions, axis = 1)

print(accuracy_score(test_set.classes,pred))

cm = confusion_matrix(test_set.classes, pred)
class_names = test_set.class_indices.keys()
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cm, classes=class_names)
plt.show()

In [ ]:
history = model.fit(train_set ,validation_data = test_set , epochs=100, batch_size=64,
                                      steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy plot")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("emotion_model_accuracy.png")

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Accuracy loss")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig("emotion_model_loss.png")

In [ ]:
predictions = model.predict(test_set)
pred = np.argmax(predictions, axis = 1)

print(accuracy_score(test_set.classes,pred))

cm = confusion_matrix(test_set.classes, pred)
class_names = test_set.class_indices.keys()
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cm, classes=class_names)
plt.show()

In [ ]:
def buildModel(dropout):
    model = Sequential([
        Input((48, 48, 1)),
        Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid'),
        BatchNormalization(axis=3),
        Activation('relu'),
        Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), strides=(1,1), padding = 'valid'),
        BatchNormalization(axis=3),
        Activation('relu'),
        Conv2D(128, (3,3), strides=(1,1), padding = 'same'),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), strides=(1,1), padding = 'valid'),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Reshape((-1,128)),
        (LSTM(128)),
        (Reshape((-1,64))),
        LSTM(64),
    #     (Reshape((-1,64)))
    #     (LSTM(32))    
        Dense(200, activation='relu'),
        Dropout(dropout),
        Dense(7, activation = 'softmax')
    ])
    model.summary()
    adam = keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam

param_grid = {
    'dropout': [0.2, 0.25, 0.5,0.6],
}

grid_search = GridSearchCV(estimator=KerasClassifier(buildModel), param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)
import pandas as pd
print(pd.DataFrame(grid_search.cv_results_))

In [ ]:
model = buildModel(0.6)
steps_per_epoch = train_set.n // train_set.batch_size
validation_steps = test_set.n // test_set.batch_size

In [ ]:
history = model.fit(train_set ,validation_data = test_set , epochs=50, batch_size=64,
                                      steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


In [ ]:
model.save("lstm.h5")

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy plot")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("emotion_model_accuracy.png")

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Accuracy loss")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig("emotion_model_loss.png")

In [ ]:
predictions = model.predict(test_set)
pred = np.argmax(predictions, axis = 1)

print(accuracy_score(test_set.classes,pred))

cm = confusion_matrix(test_set.classes, pred)
class_names = test_set.class_indices.keys()
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cm, classes=class_names)
plt.show()

74% accuracy is pretty good on this dataset. To increase accuracy further, we can experiment with regularization techniques. We'll be experimenting with more layers, and see what happens.

In [ ]:
def buildModel(activation):
    model2 = Sequential([
        Input((48, 48, 1)),
        Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid'),
        BatchNormalization(axis=3),
        Activation('relu'),
        Conv2D(64, (3,3), strides=(1,1), padding = 'same'),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), strides=(1,1), padding = 'same'),
        BatchNormalization(axis=3),
        Activation('relu'),
        Conv2D(256, (3,3), strides=(1,1), padding = 'same'),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Conv2D(256, (3,3), strides=(1,1), padding = 'same'),
        BatchNormalization(axis=3),
        Activation('relu'),
        MaxPooling2D((2,2)),
        Reshape((-1,256)),
        (LSTM(128)),
        (Reshape((-1,64))),
        LSTM(64),
        Dense(200, activation=activation),
        Dropout(0.6),
        Dense(7, activation = 'softmax')
    ])

    adam = keras.optimizers.Adam(learning_rate=0.0002)
    model.compile(optimizer=adam, loss='SparseCategoricalCrossentropy', metrics=['accuracy'])
    return model


In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam
import numpy as np
x_train, y_train = next(train_set)
x_val, y_val = next(test_set)

y_train_sparse = np.argmax(y_train, axis = 1)
y_val_sparse = np.argmax(y_val, axis = 1)

param_grid = {
    'activation': ['tanh','relu'],
}

grid_search = GridSearchCV(estimator=KerasClassifier(buildModel), param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)
import pandas as pd
print(pd.DataFrame(grid_search.cv_results_))

In [ ]:
adam = keras.optimizers.Adam(learning_rate=0.0002)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
steps_per_epoch = train_set.n // train_set.batch_size
validation_steps = test_set.n // test_set.batch_size

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adagrad, Adam

param_grid = {
    'n_untrain': [20, 40, 25, 30],
}

grid_search = GridSearchCV(estimator=KerasClassifier(buildModel), param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(x_train, y_train_sparse, epochs=2)
print("Best parameters: ", grid_search.best_params_)

In [ ]:
model2.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
history = model2.fit(train_set ,validation_data = test_set , epochs=50, batch_size=64,
                                      steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)


In [ ]:
model2.save("baseline_model.h5")

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Accuracy plot")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.savefig("emotion_model_accuracy.png")

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Accuracy loss")
plt.legend(["train","test"])
plt.xlabel("epoch")
plt.ylabel("loss")
plt.savefig("emotion_model_loss.png")

In [ ]:
predictions = model2.predict(test_set)
pred = np.argmax(predictions, axis = 1)
from sklearn.metrics import accuracy_score
print(accuracy_score(test_set.classes,pred))

In [ ]:
cm = confusion_matrix(test_set.classes, pred)
class_names = test_set.class_indices.keys()
plt.figure(figsize=(8, 6))
plot_confusion_matrix(cm, classes=class_names)
plt.show()